In [1]:
#config block

#Set to true to make the test use a siplex mesh instead of a grid
config_use_simplicies = False

Import stuff

In [2]:
import netgen.gui
%gui tk

from netgen.geom2d import SplineGeometry
from netgen.meshing import MeshingParameters
from ngsolve import *
from xfem import *
# For LevelSetAdaptationMachinery
from xfem.lsetcurv import *
from make_uniform2D_grid import * 
import types

import numpy as np

from scipy.sparse.linalg import inv
from scipy import sparse as sp
from functools import wraps # This convenience func preserves name and docstring


In order to test  this document's method for different mesh sizes, we put everything in methods of a 'SquareTest' class.

In [3]:
#our 'class', but scattered over this jupyter notebook. 
class SquareTest(object):
    pass

def add_method(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func # returning func means func can still be used normally
    return decorator


Setup the Mesh

In [4]:

st = SquareTest()

@add_method(SquareTest)
def create_mesh(self, N):
    self.mesh = MakeUniform2DGrid(quads=True,N=N,P1=(-1,-1),P2=(1,1))
    if config_use_simplicies:
        #test for simplicies
        square = SplineGeometry()
        square.AddRectangle([-1,-1],[1,1],bc=1)
        self.mesh = Mesh(square.GenerateMesh(maxh=0.25, quad_dominated=False))
        
st.create_mesh(10)
Draw(st.mesh)

helper functions

In [5]:
# helper function we need later to calculate the conditional number.
# @param A : a scipy.sparse matrix
def linalg_cond(A):
    norm_A = sp.linalg.norm(A)
    norm_invA = sp.linalg.norm(sp.linalg.inv(A))
    return norm_A*norm_invA

#shows draws an image with the given Vertex highlighted
def ShowGlobalVertex(i):
    gf_v = GridFunction(H1(self.mesh))
    gf_v.vec[:]=0; gf_v.vec[i]=1
    Draw(gf_v,st.mesh,"current_vertex")
    print("Drawing vertex",i)

#shows draws an image with the given Element highlighted
def ShowElement(i):
    gf_e = GridFunction(L2(st.mesh))
    gf_e.vec[:]=0; gf_e.vec[i]=1
    Draw(gf_e,st.mesh,"current_element")
    print("Drawing element",i)   


Setting up the test problem, we will get a liniar problem of the form
\begin{align}
A g = f
\end{align}
Where the colums&rows of A,f and g correpond to the degrees of freedom out H1 space which are in 1:1 correpsondance to the verticies in our mesh, after this we no longer need the detail of the problem we just work with the matricies A, f and the mesh information about the cut info.

In [6]:
######### SETUP of "naive" problem #########

@add_method(SquareTest)
def create_problem(self):
    order = 1

    R=0.8
    levelset = 1*(sqrt(x**2+y**2)-R)
    coeff_f = 10
    exact = 2.5*(R**2 - (x**2+y**2))
    # stabilization parameter for Nitsche
    lambda_nitsche  = 100

    h = specialcf.mesh_size   

    lsetp1 = GridFunction(H1(self.mesh))
    lsetp1.Set(levelset)
    n_levelset = 1.0/Norm(grad(lsetp1)) * grad(lsetp1)

    lset_neg = { "levelset" : lsetp1, "domain_type" : NEG, "subdivlvl" : 0}
    lset_if  = { "levelset" : lsetp1, "domain_type" : IF , "subdivlvl" : 0}

    # element, facet and dof marking w.r.t. boundary approximation with lsetp1:
    ci = CutInfo(self.mesh,lsetp1)

    Vh = H1(self.mesh, order = order, dirichlet=[])
    #Vh = Compress(Vh,active_dofs)
    #active_dofs = GetDofsOfElements(Vh,ci.GetElementsOfType(HASNEG))


           
    a = BilinearForm(Vh,symmetric=False)
    f = LinearForm(Vh)
            
    u,v = Vh.TrialFunction(), Vh.TestFunction()

    # Diffusion term
    a += SymbolicBFI(lset_neg,form = grad(u)*grad(v), definedonelements=ci.GetElementsOfType(HASNEG))
    # Nitsche term
    nitsche_term  = -grad(u) * n_levelset * v
    nitsche_term += -grad(v) * n_levelset * u
    nitsche_term += (lambda_nitsche/h) * u * v
    a += SymbolicBFI(lset_if, form = nitsche_term, definedonelements=ci.GetElementsOfType(IF))
    # rhs term:
    f += SymbolicLFI(lset_neg, form=coeff_f*v, definedonelements=ci.GetElementsOfType(HASNEG))


    # ba_facets = GetFacetsWithNeighborTypes(mesh,a=ci.GetElementsOfType(HASNEG),b=ci.GetElementsOfType(IF))
    # facets on which ghost penalty stabilization should be applied
    # a += SymbolicFacetPatchBFI(form = 0.1*1.0/h*1.0/h*(u-u.Other())*(v-v.Other()),
    #                            skeleton=False,
    #                            definedonelements=ba_facets)


    a.Assemble()
    f.Assemble()
    self.Vh = Vh
    #the A g = f equation is what we will later reusce to a smaller equation with a better condition number.
    self.a = a
    self.f = f
    #we need this information later to compare our calculated functions to this, to compute the error.
    self.exact = exact
    #we only need this to draw it.
    self.levelset = levelset
    #the cut information, the main ingridient for out later algothirm.
    self.ci = ci
    self.lset_neg = lset_neg
    self.lset_if = lset_if
    self.active_dofs = GetDofsOfElements(Vh, ci.GetElementsOfType(HASNEG))
    self.inner_dofs = GetDofsOfElements(Vh, ci.GetElementsOfType(NEG))

st.create_problem()
Draw(st.levelset, st.mesh, "levelset")


Solve the problem with the old method (just solving the $A g= f$ liniear equation) 

In [7]:
def norm1(a):
    A_rows,A_cols,A_vals = a.mat.COO()
    return sp.linalg.norm(sp.csr_matrix((A_vals,(A_rows,A_cols))))

# linalg_cond as defined as above does not work with BilinearForm objects so we need this.

@add_method(SquareTest)
def cond1(st):
    #A = sp.lil_matrix((st.active_dofs.NumSet(), st.active_dofs.NumSet()))
    A = sp.csr_matrix((st.active_dofs.NumSet(), st.active_dofs.NumSet()))
    i2 = 0
    #remove rows with zeroes to make linalg_cond happy (obviously this code could be optimized)
    for i in range(len(st.inner_dofs)):
        if st.active_dofs[i]:
            j2 = 0
            for j in range(len(st.inner_dofs)):
                if st.active_dofs[j]:
                    if st.a.mat[(i, j)] != 0:
                        A[(i2, j2)] = st.a.mat[(i, j)]
                    j2 = j2 + 1
            i2 = i2 + 1
    #return linalg_cond(A.tocsc())
    return linalg_cond(A.tocsc())
    

@add_method(SquareTest)
def old_method(self):
    gfu = GridFunction(st.Vh)
    gfu.vec.data = st.a.mat.Inverse(st.active_dofs) * st.f.vec
              
    l2error = sqrt(Integrate(st.lset_neg,(gfu-st.exact)*(gfu-st.exact), st.mesh))
    print("error:", l2error)
    #Draw(lsetp1,mesh,"lsetp1")
    #Draw(gfu,mesh,"extu")
    #Draw(IfPos(-lsetp1,gfu,float('nan')),mesh,"u",sd=5)
    print("norm 'a':", norm1(st.a))
    print("conditional number:", cond1(st))
    return gfu

print("Dofs: inner=" + str(st.inner_dofs.NumSet()) + " active:" + str(st.active_dofs.NumSet()))
gfu = st.old_method()

Dofs: inner=45 active:89
error: 0.02674610577984627
norm 'a': 263.471660322


/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


conditional number: 271008880.852


Build the clusters using the alghorithm

In [8]:
from ad_info import *
from cluster_info import *

st.ad = AdInfo(st.mesh)
st.cluster_info = CusterInfo(st.ad, st.ci)

In [9]:
Draw(BitArrayCF(st.ci.GetElementsOfType(IF)), st.mesh, "cut_elements")

Tests the Function to check adjaceny of two elements.

In [10]:
st.ad.test_f2f(3)

Draws All the Clusters

In [11]:
st.cluster_info.draw_clusters()

In [12]:
from last_step import *

`coefficient_calc` can now be used to calculate the coefficient t of the matrix C below

In [13]:
st.coefficient_calc = LastStep(st.mesh, st.ci)

Testing `coefficient_calc`, currently only works for grids.

In [14]:
# not really useful c_fix_row_test below can alos be use instead.
if False and not config_use_simplicies:
    ci_neg = st.ci.GetElementsOfType(NEG)
    ci_if = st.ci.GetElementsOfType(IF)
    ci_pos = st.ci.GetElementsOfType(POS)

    for v in st.mesh.vertices:
        adf = ad.get_vertex_ajacent_faces(v.nr)
        #If v is adjacent to an inner cell we do not need to calculate the corresponding matrix row (it is the identity) 
        if not any(ci_neg[f_id] for f_id in adf):
            ad_face = next( (f_id for f_id in adf if ci_if[f_id]), None)
            #If v is not adjacent to a cut cell we do not need to calculate the corresponding matrix row (it is zero)
            if not ad_face is None:
                print(*coefficient_calc.do_squares_2D(v.nr, st.cluster_info.get_interpolation_face(v.nr)), sep=",")


We restrict ourself to such solutions where the coefficents associated to the verticies that are not incident to an inner face are complteley determined by the coeffients on associated to the inner Vertices. The correspondance is descriped via a matrix $C$ such that when $b$ is the vector if inner coefficents that full vector fo coefficnets can be descripbed as $C  \dot b$
So the linear equation we have to solve in the end becomes
\begin{align}
C^t A C g = C^t f
\end{align}
$C$ is a matrix fo size Num_Dofs $\times$ num_inner_dofs so in order to make useful things with the inner dof indicies we need a dictionary to convert these indicies.

`dof_to_innerdof` maps the dof ids to the restricted list for inner dofs. This is needed to calculate which column in the matrix `C` below corresponds to which dof of our Function space.
`innerdof_to_dof` is the reverse map, `dof_to_innerdof` is -1 for verticies that do not correpsind to inner dofs.

In [15]:
@add_method(SquareTest)
def setup_dof_convert1(self):
    self.dof_to_innerdof = [0 for i in range(self.Vh.ndof)]
    self.innerdof_to_dof = []
    for i, inner in enumerate(self.inner_dofs):
        if inner:
            self.dof_to_innerdof[i] = len(self.innerdof_to_dof)
            self.innerdof_to_dof.append(i)
        else:
            self.dof_to_innerdof[i] = -1
    self.n_innerdofs = len(self.innerdof_to_dof)
    self.n_dofs = self.Vh.ndof

st.setup_dof_convert1()
@add_method(SquareTest)
def dof_convert(self, i):
    return self.dof_to_innerdof[i]

Setup a list to map verticies to the assiciated Dofs ( = column/row numbers of the matrix a). In our case this will just end up being the identily map (`[1,2,3, ...]`), but i do not think the library guarantees that so its better to do this.

the maps `dof_to_vertex` and `vertex_to_dof` are inverse to each other

In [16]:
@add_method(SquareTest)
def setup_dof_convert2(self):
    self.dof_to_vertex_list = [None for i in range(self.Vh.ndof)]
    for v in self.mesh.vertices:
        self.dof_to_vertex_list[self.Vh.GetDofNrs(v)[0]] = v.nr
st.setup_dof_convert2()
@add_method(SquareTest)
def dof_to_vertex(self, d_id):
    return self.dof_to_vertex_list[d_id]
@add_method(SquareTest)
def vertex_to_dof(self, d_id):
    return self.Vh.GetDofNrs(NodeId(VERTEX,d_id))[0]

Now we have to tools to create the Matrix $C$, its rows can be categorizes as 3 types:

If the row i corresponds to an outer vertex we have a row full of zeroes.

If the row i corresponds to an inner vertex we have a zero row except at position (i,i) where we ahve a 1 entry.

If the row i corresponds to an aggregate vertex we have to use coefficient_calc.get_coefficients to calculate the matrix entries.

In [17]:

#row_index must correspond to an aggreage vertex
@add_method(SquareTest)
def c_fix_row(self, row_index, C):
    v_id = self.dof_to_vertex(row_index)
    # print('v_id', row_index, v_id)
    coefcients = self.coefficient_calc.get_coefficients(v_id, self.cluster_info.get_interpolation_face(v_id))
    for c in coefcients:
        dof = self.vertex_to_dof(c.vertex_id)
        # print('dof', c.vertex_id, dof)
        C[row_index, self.dof_convert(dof)] = c.coefficient

        
@add_method(SquareTest)
def create_C(self):
    C = sp.lil_matrix((self.n_dofs, self.n_innerdofs)) #(rows, columns)
    for i in range(self.n_dofs):
        if self.inner_dofs[i]:
            #print(i, "is inner")
            C[i, self.dof_convert(i)] = 1
        elif self.active_dofs[i]:
            #print(i, "is active")
            self.c_fix_row(i, C)
    return C

C = st.create_C()

Solving the linear Equation
\begin{align}
C^t A C g = C^t f
\end{align}
and store the resulting function in the `gfu2` variable. calculate the error and the conditional number and compare it to the original. The error is a bit worse in our example but the conditional number should be much better.

In [18]:
@add_method(SquareTest)
def new_method(self, C):
    C_t = C.transpose()
    A_rows,A_cols,A_vals = self.a.mat.COO()
    A = sp.csr_matrix((A_vals,(A_rows,A_cols)))
    F = self.f.vec.FV().NumPy().copy()

    A2 = C_t * A * C
    F2 = C_t * F

    gfu2 = GridFunction(self.Vh)
    #gfu2.vec.data = a.mat.Inverse(active_dofs) * f.vec
    data = C * inv(A2) * F2
    for i in range(self.n_dofs):
        #if inner_dofs[i]:
        #    gfu2.vec[i] = data[dof_convert(i)]
        gfu2.vec[i] = data[i]
    #print(gfu2.vec)

    l2error_2 = sqrt(Integrate(st.lset_neg,(gfu2-st.exact)*(gfu2-st.exact),st.mesh))

    print("error new:", l2error_2)
    print("norm 'a' new:", sp.linalg.norm(A2))
    print("conditional number new:", linalg_cond(A2))
    return gfu2
gfu2 = st.new_method(C)
print("ndofs", st.n_dofs, "ninnerdofs", st.n_innerdofs)

error new: 0.0807476565049812
norm 'a' new: 1578.66199937
conditional number new: 5892.10726404
ndofs 121 ninnerdofs 45


/usr/lib/python3/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:253: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/usr/lib/python3/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:171: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


Draw the results

In [19]:
#The generated function using the simple old method
Draw(gfu, st.mesh, "original_gfu")
#The generated function using the new method
Draw(gfu2, st.mesh, "gfu2")
#The exact solution
Draw(st.exact, st.mesh, "exact")


Test (print) the result of `coefficient_calc.get_coefficients`, NOTE: this assumes row 55 corersponds to a agrgate vertex which is in particular the case for a 10 $\times$ 10 square grid, (meaning: this might fail of you changed parts of the grid setup above) 

In [20]:
def c_fix_row_test(row_index, C):
    v_id = st.dof_to_vertex(row_index)
    print('vertex_id', v_id, "row", row_index)
    coefcients = st.coefficient_calc.get_coefficients(v_id, st.cluster_info.get_interpolation_face(v_id))
    print("face_id", st.cluster_info.get_interpolation_face(v_id))
    for c in coefcients:
        dof = st.vertex_to_dof(c.vertex_id)
        print("vertex:", c.vertex_id, "   dof:", dof, "   coeff:", c.coefficient, "    coll:", st.dof_convert(dof))
c_fix_row_test(55, C)


vertex_id 55 row 55
face_id 42
vertex: 46    dof: 46    coeff: -0.0     coll: 12
vertex: 47    dof: 47    coeff: 0.0     coll: 13
vertex: 58    dof: 58    coeff: -2.0000000000000004     coll: 20
vertex: 57    dof: 57    coeff: 3.0000000000000004     coll: 19


Now Test the whole thing for differnt grid sizes

In [21]:
NValues = [8,10,15,20,30]
for N in NValues:
    print("\n\nN =", N)
    st = SquareTest()
    st.create_mesh(N)
    st.create_problem()
    st.old_method()
    st.ad = AdInfo(st.mesh)
    st.cluster_info = CusterInfo(st.ad, st.ci)
    st.coefficient_calc = LastStep(st.mesh, st.ci)
    st.setup_dof_convert1()
    st.setup_dof_convert2()
    C = st.create_C()
    st.new_method(C)



N = 8
error: 0.04143652481890676
norm 'a': 239.491826961
conditional number: 39188678.2157
error new: 0.09653116984338242
norm 'a' new: 728.090481247


/usr/lib/python3/dist-packages/scipy/sparse/compressed.py:774: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)
/usr/lib/python3/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:253: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/usr/lib/python3/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:171: SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
  'is in the CSC matrix format', SparseEfficiencyWarning)


conditional number new: 1830.67606256


N = 10
error: 0.02674610577984627
norm 'a': 263.471660322
conditional number: 271008880.852
error new: 0.0807476565049812
norm 'a' new: 1578.66199937
conditional number new: 5892.10726404


N = 15
error: 0.01205543266642842
norm 'a': 289.130461704
conditional number: 39199.6306497
error new: 0.03203005598614145
norm 'a' new: 1637.24042548
conditional number new: 13797.1226818


N = 20
error: 0.006842264969305606
norm 'a': 372.730027852
conditional number: 1031184798.0
error new: 0.014049966991262576
norm 'a' new: 1981.59362383
conditional number new: 29285.0909377


N = 30
error: 0.0030624330495531847
norm 'a': 452.084502242
conditional number: 1114142402.47
error new: 0.005649642789006835
norm 'a' new: 2963.00112465
conditional number new: 96857.1549175
